# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pratikmurali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pratikmurali/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  92995      0 --:--:-- --:--:-- --:--:-- 92839


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   182k      0 --:--:-- --:--:-- --:--:--  183k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 11, relationships: 37)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 11, relationships: 37)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


#### 📍 Answer #1

<div style="color: blue; font-family: Arial, sans-serif;">

**SingleHopSpecificQuerySynthesizer:** Creates direct questions that can be answered from a single piece of information in the docs. These are focused questions that typically ask about specific facts, definitions, or details contained in one location in the text

**Examples:**
- "What is midjourney?"
- "Why is 2023 considered a breakthrough year for AI?"
- "What is the significance of Abu Dhabi in the context of AI?"

**MultiHopAbstractQuerySynthesizer:** Generates broader, conceptual questions that require synthesizing information from multiple parts of the loaded docs. These questions are more theoretical and ask about relationships, impacts, or high-level concepts that span across different sections of content.

**Examples:**
- "How does the environmental impact of AI training affect the future development of models?"
- "How can prompt engineering influence the effectiveness of language models?"
- "How large language models (LLMs) are related to future AI developments?"

**MultiHopSpecificQuerySynthesizer:** Creates complex questions that require connecting specific details from different parts of the loaded docs. These combine the specificity of single-hop queries with the multi-source nature of abstract queries, asking about detailed relationships between specific entities or concepts.

**Examples:**
- "How does Llama 3.1 relate to the development of open-source AI models?"
- "What is Claude's role in integrating AI features across different platforms?"
- "How do LLMs impact AI development and how are they so advanced?"

Each synthesizer is given a weight in the distribution (0.5, 0.25, 0.25) to control how many of each type appear in the generated test dataset, creating a balanced mix of question complexity to better evaluate RAG systems

</div>

Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What are LLMs and why are they significant?,[The ethics of this space remain diabolically ...,Large Language Models (LLMs) are software syst...,single_hop_specifc_query_synthesizer
1,Who Simon Willison,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison’s Weblog discusses that 2023 wa...,single_hop_specifc_query_synthesizer
2,What are LLMs and why are they significant in ...,[the document includes some of the clearest ex...,Large language models (LLMs) are a central top...,single_hop_specifc_query_synthesizer
3,"As an AI research scientist, how does OpenAI's...",[The year of slop Synthetic training data work...,"The context indicates that in 2024, OpenAI is ...",single_hop_specifc_query_synthesizer
4,How does ChatGPT's voice mode enhance user int...,[also pre-announced voice mode for Amazon Nova...,ChatGPT voice mode now provides the option to ...,single_hop_specifc_query_synthesizer
5,How do AI capabilities like question answering...,[<1-hop>\n\nThe ethics of this space remain di...,Recent developments have shown that large lang...,multi_hop_abstract_query_synthesizer
6,how AI training impact environment and is it b...,[<1-hop>\n\nI would be very surprised if they ...,The context explains that the energy usage and...,multi_hop_abstract_query_synthesizer
7,How access to AI models and their avilability ...,[<1-hop>\n\nalso pre-announced voice mode for ...,The context explains that for a few short mont...,multi_hop_abstract_query_synthesizer
8,How do Llama 3.1 and Llama 3.3 70B models comp...,[<1-hop>\n\nthe importance of synthetic data. ...,The context indicates that Meta's Llama 3.3 70...,multi_hop_specific_query_synthesizer
9,Hw do LLMs work and why are they so easy to bu...,[<1-hop>\n\nThe ethics of this space remain di...,The document explains that LLMs are built usin...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is TII and how has it contributed to the ...,[The ethics of this space remain diabolically ...,"TII, based in Abu Dhabi, is one of the organiz...",single_hop_specifc_query_synthesizer
1,What is the relationship between Microsoft and...,"[and software engineer, LLMs are infuriating. ...",The provided context does not specify the rela...,single_hop_specifc_query_synthesizer
2,Considering Simon Willison’s insights from his...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison’s weblog highlights that 2023 w...,single_hop_specifc_query_synthesizer
3,What role does OpenAI play in the development ...,[the document includes some of the clearest ex...,The context mentions OpenAI as one of the key ...,single_hop_specifc_query_synthesizer
4,How do the increased context lengths and input...,[<1-hop>\n\nThe rise of inference-scaling “rea...,The Phi-4 technical report emphasizes that syn...,multi_hop_abstract_query_synthesizer
5,how can prompt engineering and injection be us...,[<1-hop>\n\nOpenAI aren’t the only group with ...,The context describes how recent advancements ...,multi_hop_abstract_query_synthesizer
6,How do misconceptions about AI and the gullibi...,"[<1-hop>\n\nagain, on the challenge of gullibi...",The context highlights that LLMs tend to belie...,multi_hop_abstract_query_synthesizer
7,How do the recent developments in multi-modal ...,[<1-hop>\n\non a story about the town's histor...,"Recent advancements in multi-modal AI models, ...",multi_hop_abstract_query_synthesizer
8,How do the complex legal and ethical considera...,[<1-hop>\n\nthe document includes some of the ...,The context highlights that the legal argument...,multi_hop_specific_query_synthesizer
9,How do Apple’s MLX library and Apple Intellige...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"According to the context, Apple’s MLX library ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years - Part Deux!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years - Part Deux!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI - Part Deux!"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term with no single, clear, and widely understood meaning. Generally, there are two main interpretations:\n\n1. AI systems that go and act on your behalf, similar to a travel agent model.\n2. Large language models (LLMs) given access to tools which they can use iteratively to solve problems.\n\nThe term is often associated with "autonomy," but without a clear definition. Despite being widely discussed and anticipated, agents have not yet been realized in meaningful, production-level applications. They remain a concept that is perpetually "coming soon."\n\nAdditionally, the usefulness of agents is questioned due to the challenge of gullibility—LLMs tend to believe whatever they are told, which limits an agent\'s ability to make reliable decisions on behalf of a user. Fully solving gullibility may require artificial general intelligence (AGI), making true autonomous agents a distant goal.\n\nIn summary, "agents" refers to

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

#### 💡 Solution Activity #2:

<div style="color: blue; font-family:sans-serif;">
The three evaluators are assessing different aspects of LLM responses:
<ul>
<li> <b> qa_evaluator:</b> Evaluates the correctness of answers by comparing them against reference answers. This built-in evaluator measures factual accuracy and completeness of responses to questions.</li>
<li> <b> labeled_helpfulness_evaluator:</b> Assesses how helpful the response is to the user, considering the reference answer as ground truth. The custom prepare_data function maps the model's output, the expected answer, and the original question for proper evaluation.</li>
<li> <b> dope_or_nope_evaluator:</b> A subjective evaluator that measures the "coolness" or stylistic appeal of responses - whether they're engaging, interesting, or appealing in tone. This is a less standard metric focused on user experience rather than factual correctness.
</li>
</ul>
Each evaluator uses the same LLM (eval_llm) to perform these assessments, leveraging it as a judge of the generated content.
</div>


## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'pertinent-sheet-64' at:
https://smith.langchain.com/o/897469e2-b134-4dc6-8475-ea3514973522/datasets/e03314aa-7a19-442d-8e99-30223e07367c/compare?selectedSessions=c0e505fb-fbbd-4a7b-a569-43748fb948e0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do the themes of Llama and Llama 2 relate ...,The themes of Llama and Llama 2 relate to the ...,None,The context segments highlight significant pro...,1,1,0,4.578360,72d75f4b-a825-402a-b663-927f8a46472c,a65d3017-384d-4bd4-8ee4-1fd981d46d2f
1,How do the developments in Llama 2 and Llama 3...,The developments in Llama 2 and Llama 3.1 illu...,None,The context highlights significant progress in...,1,1,0,8.290508,379a8e71-a03e-4b81-8256-a054de17230f,cfbda48f-1d38-45ea-bb76-accd8700577a
2,How do Apple’s MLX library and Apple Intellige...,"Based on the provided context, Apple’s MLX lib...",None,"According to the context, Apple’s MLX library ...",1,1,0,6.865152,ab12d552-c1fe-43e3-962e-bf1e92d1b1d2,324ada45-0551-48c4-a7b1-bbdeef4174e1
3,How do the complex legal and ethical considera...,The context indicates that the legal arguments...,None,The context highlights that the legal argument...,1,1,0,3.703110,ed06d0b9-6022-438e-8d74-d07dcc2654e6,b4e231f0-2d98-44d3-93c5-5067b0e077f0
4,How do the recent developments in multi-modal ...,Recent developments in multi-modal AI models h...,None,"Recent advancements in multi-modal AI models, ...",1,1,0,6.114835,50097dc4-1f7e-4999-b867-69404c8d2c3d,fe9ea50d-5e30-4efa-8bd5-768266267d0a
5,How do misconceptions about AI and the gullibi...,"Based on the provided context, misconceptions ...",None,The context highlights that LLMs tend to belie...,1,0,0,5.633590,e2fabfe2-67da-4966-a56a-a1ef6c2db39b,e6722da8-17f6-456e-9bcb-2fa8e269da80
6,how can prompt engineering and injection be us...,I don't know.,None,The context describes how recent advancements ...,0,0,0,1.393685,5e4ae7a5-7ac5-4d6c-a2e4-1d684d7feaa1,cb2ad1c4-7377-46fb-aa7f-57efc03eedda
7,How do the increased context lengths and input...,"Based on the provided context, increased conte...",None,The Phi-4 technical report emphasizes that syn...,1,1,0,5.113304,8e81c5ab-772f-41e5-921b-eab6db274e89,20397119-d859-4964-bde3-db6bf8ea37d8
8,What role does OpenAI play in the development ...,OpenAI played a pioneering role in the develop...,None,The context mentions OpenAI as one of the key ...,1,1,0,2.797491,41768a60-f87b-4c19-b3e4-5827abbfac65,dcab337b-2c0d-49f2-a1ab-64839e6ddabe
9,Considering Simon Willison’s insights from his...,Based on the provided context from Simon Willi...,None,Simon Willison’s weblog highlights that 2023 w...,1,1,0,6.043943,d7b9f92a-fdee-4a84-933d-fe419d723114,7e532c74-75d6-4d90-b172-0794b5b4af7d


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### 📍 Answer #2:
<div style="color: blue; font-family:sans-serif;">

Increasing the chunk size from 500 to 1000 characters likely improves performance by providing more complete context to the retrieval system, which helps the model generate more accurate and comprehensive responses, especially for complex or nuanced questions. Smaller chunks provide more focused and precise retrieval that can target specific information, but might miss broader context
</div>

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### 📍 Answer #3:

<div style="color: blue; font-family:sans-serif;">

Modifying the embedding model from "text-embedding-3-small" to "text-embedding-3-large" will impact RAG performancein several ways:
<ul>
<li> Better Semantic Understanding: because the large model offers higher dimension embeddings with more nuanced semantics.</li>
<li> Better Retrieval Accuracy: It allows for more precise similarity calculations between queries and doc chunks. Will also reduce false positives </li>
<li> Better handling of Contextual Complexity: It can provide more robust connection between related technical terms and improved ability to match context in different ways.</li>
<li> Better Contextual Awareness: Beter performance with abstract or complex queries and better handling of ambiguous terms based on surrounding context.</li>
<li> Better Language Nuance : Larger embedding models can provide better handling of domain specific language and are more effective with indirect or inferential queries.
</ul>

</div>


In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Yo, here’s the lowdown on “agents” straight from the context: \n\nAgents are kinda like these AI things that people hype but don't really nail down — the term is super vague. Some folks think of agents as AI that can go off and do stuff for you, like a travel agent booking trips on the fly. Others see agents as large language models (LLMs) hooked up with tools, looping through tasks to solve problems. But here’s the catch: no one’s settled on what “autonomy” really means for these agents.\n\nDespite all the buzz, real-deal agents that act independently and reliably haven’t really landed in the wild yet. The major snag? Gullibility. These AIs tend to believe whatever they hear, which makes them shaky when you want them to make smart decisions on your behalf. Serious agency needs something like AGI to handle that trustworthiness, and we’re not quite there yet.\n\nSo yeah, agents are these hyped-up, kinda fuzzy AI systems meant to do stuff for you, but right now they’re mostly “coming so

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'earnest-sheet-85' at:
https://smith.langchain.com/o/897469e2-b134-4dc6-8475-ea3514973522/datasets/e03314aa-7a19-442d-8e99-30223e07367c/compare?selectedSessions=bb35e60b-1d75-421d-ade8-09608908d2d5




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do the themes of Llama and Llama 2 relate ...,"Ah, here’s the lowdown: Llama and Llama 2 vibe...",None,The context segments highlight significant pro...,1,1,1,5.690520,72d75f4b-a825-402a-b663-927f8a46472c,58379bfe-7e9c-4865-bf0d-c039ff9a4351
1,How do the developments in Llama 2 and Llama 3...,"Yo, here’s the lowdown with Llama 2, Llama 3.1...",None,The context highlights significant progress in...,1,1,1,5.629988,379a8e71-a03e-4b81-8256-a054de17230f,d54bd6d7-a9f8-4fd0-a8f5-ebd040cc449d
2,How do Apple’s MLX library and Apple Intellige...,"Yo, here’s the lowdown: Apple’s MLX library is...",None,"According to the context, Apple’s MLX library ...",1,1,1,7.515837,ab12d552-c1fe-43e3-962e-bf1e92d1b1d2,40f04948-2e22-49ad-a450-dff931b0f116
3,How do the complex legal and ethical considera...,"Yo, here’s the lowdown: The legal and ethical ...",None,The context highlights that the legal argument...,1,1,1,4.617130,ed06d0b9-6022-438e-8d74-d07dcc2654e6,a28fec37-45c5-4bfe-a720-b2768e872787
4,How do the recent developments in multi-modal ...,"Yo, the multi-modal AI wave in 2024 is straigh...",None,"Recent advancements in multi-modal AI models, ...",1,1,1,5.079055,50097dc4-1f7e-4999-b867-69404c8d2c3d,a38b9aaa-6f9d-4f4a-b919-2ee73e83cb97
5,How do misconceptions about AI and the gullibi...,"Alright, here’s the lowdown, straight from the...",None,The context highlights that LLMs tend to belie...,1,0,1,8.404440,e2fabfe2-67da-4966-a56a-a1ef6c2db39b,8f6ffb68-6422-4cd5-ac85-5dbb547e0730
6,how can prompt engineering and injection be us...,"Yo, here’s the lowdown on how prompt engineeri...",None,The context describes how recent advancements ...,1,1,1,7.226803,5e4ae7a5-7ac5-4d6c-a2e4-1d684d7feaa1,d870a43d-d652-4088-88e7-5361a095fcac
7,How do the increased context lengths and input...,"Yo, here’s the lowdown straight from the conte...",None,The Phi-4 technical report emphasizes that syn...,1,1,1,12.700302,8e81c5ab-772f-41e5-921b-eab6db274e89,9721e15d-5151-43c4-a4bf-6b41593f251f
8,What role does OpenAI play in the development ...,"Yo, OpenAI’s the big boss in the LLM game, dro...",None,The context mentions OpenAI as one of the key ...,1,1,1,3.374776,41768a60-f87b-4c19-b3e4-5827abbfac65,63866960-f849-48d4-bc87-8f1afa586fb0
9,Considering Simon Willison’s insights from his...,"Yo, here’s the lowdown straight from Simon Wil...",None,Simon Willison’s weblog highlights that 2023 w...,1,1,1,3.685155,d7b9f92a-fdee-4a84-933d-fe419d723114,362db56c-7a11-451c-9f64-a2c98db0d971


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### 💡 Solution #3

The dope_rag_prompt which we added to the dopeness LCEL chain, has resulted in the dopeness factor of the answers evaluating better.

<img src = "images/Langsmith_Experiment.png" height = "500" width="900">